In [ ]:
import pandas as pd
from smartsheet_dataframe import get_as_df
import os

In [ ]:
# Storing Smartsheet information in variables
shitz_token = "wMvGniITjDFd7ClZPE44wtJGvZkM7Hg7mP7if" #Bily's personal token to access G-A smartsheet data. Anyone can obtain their own token
fhaid = 4523924143794052 # The ID number that references the FHA sheet
mssid = 3005704744265604 # The ID number that references the MSS sheet

# Creating the FHA and MSS dataframes
fha_df = get_as_df(type_='sheet',
                  token=shitz_token,
                  id_=fhaid)

mss_df = get_as_df(type_='sheet',
                  token=shitz_token,
                  id_=mssid)

data_df = mss_df.loc[(mss_df['UniqueID'].str.contains('WAR')) |
                     (mss_df['UniqueID'].str.contains('WRR')) |
                     (mss_df['UniqueID'].str.contains('BJT'))].copy()

data_df['Tract'] = data_df['Tract'].astype(str).apply(lambda x: x.split('.')[0])
data_df['CMP'] = data_df['CMP'].astype(str).apply(lambda x: x.split('.')[0])
data_df['Stand'] = data_df['Stand'].astype(str).apply(lambda x: x.split('.')[0])
data_df['TCS'] = data_df['Tract'] + "_" + data_df['CMP'] + "_" + data_df['Stand']

data_df['Reporting Period'] = data_df['Reporting Period'].fillna(0)
data_df.loc[data_df['Reporting Period'] == "", 'Reporting Period'] = 0
s = data_df.loc[pd.to_numeric(data_df['Reporting Period'], errors='coerce').isnull() & data_df['Reporting Period'].str.contains("RP"), 'Reporting Period'].apply(lambda x: x.strip()[-1])
data_df['trial'] = data_df.index.map(s)
data_df.loc[pd.to_numeric(data_df['Reporting Period'], errors='coerce').isnull() & data_df['Reporting Period'].str.contains("RP"), 'Reporting Period'] = data_df.loc[pd.to_numeric(data_df['Reporting Period'], errors='coerce').isnull() & data_df['Reporting Period'].str.contains("RP"), 'trial']
data_df['Reporting Period'] = pd.to_numeric(data_df['Reporting Period'], downcast='integer')


## Creaet Client Table

In [ ]:
# Filter out the Client Provided data and make sure the fields are similar
client_df = data_df[['UniqueID',
                    'Tract',
                    'CMP',
                    'Stand',
                    'Acres',
                    'Age',
                    'County',
                    'State',
                    'Stand Type',
                    'Origin',
                    'Scheduled',
                    'PreBA',
                    'PreTPA',
                    'PreGTA',
                    'PlnBA',
                    'PlnTPA',
                    'PlnGTA',
                    'Client GT',
                    'Future GT',
                    'Reporting Period']].copy()

client_df['Age'] = pd.to_numeric(client_df['Age'], errors='coerce', downcast='integer').fillna(0).astype(int)
client_df.loc[pd.to_numeric(client_df['Future GT'], errors='coerce').isnull(), 'Future GT'] = 0
client_df['Tract'] = client_df['Tract'].astype(str)
client_df['Stand'] = client_df['Stand'].astype(str)
client_df['Tract'] = client_df['Tract'].apply(lambda x: x.split('.')[0])
client_df['Client GT'] = pd.to_numeric(client_df['Client GT'], errors="coerce")
client_df['Client GT'] = client_df['Client GT'].fillna(0)

cols = ['PreBA', 'PreTPA', 'PreGTA', 'PlnBA', 'PlnTPA', 'PlnGTA']

for x in cols:
    client_df[x] = pd.to_numeric(client_df[x], errors='coerce', downcast='integer').fillna(-99).astype(int)



# Normalize some of the fields in the client intake table

## Normalize Species
species = ["Slash Pine", "Loblolly Pine", 'Mixed Natural Pine', 'Hardwood Mix', 'Pine Straw', "Other"]
species_dict = {'Species': species}
species_df = pd.DataFrame(species_dict)
species_df['SpeciesUID'] = species_df.index

def species_map(x):
    x = x.lower()
    if "slash" in x:
        return 0
    elif "loblol" in x:
        return 1
    elif "mixed natural" in x:
        return 2
    elif "h" in x:
        return 3
    elif "straw" in x:
        return 4
    elif "pine" in x:
        return 1
    else:
        return 5
    
client_df['Stand Type'] = client_df['Stand Type'].apply(lambda x: species_map(x))


## Normalize origin
origins = ['Planted', 'Natural', 'Natural Mix', 'Other']
origin_dict = {'Origin': origins}
origin_df = pd.DataFrame(origin_dict)
origin_df['OriginUID'] = origin_df.index

def origin_map(x):
    x = x.lower()
    if "plant" in x:
        return 0
    elif "natural mix" in x:
        return 2
    elif "nat" in x:
        return 1
    else:
        return 3
client_df['Origin'] = client_df['Origin'].apply(lambda x: origin_map(x))


## Normalize tract
client_df['tract_l'] = client_df['Tract'].apply(lambda x: x.lower())
tracts = client_df['tract_l'].unique().tolist()
tract_dict = {'tract': tracts}
tract_df = pd.DataFrame(tract_dict)
tract_df['tract_oid'] = tract_df.index
new_tract_dict = dict(zip(tract_df['tract'], tract_df['tract_oid']))
client_df['Tract'] = client_df['tract_l'].map(new_tract_dict)

## Normalize county
client_df['County'] = client_df['County'].astype(str)
client_df['county_l'] = client_df['County'].apply(lambda x: x.lower())
countys = client_df['county_l'].unique().tolist()
county_dict = {'county': countys}
county_df = pd.DataFrame(county_dict)
county_df['county_oid'] = county_df.index
new_county_dict = dict(zip(county_df['county'], county_df['county_oid']))
client_df['County'] = client_df['county_l'].map(new_county_dict)

## Normalize state
client_df['State'] = client_df['State'].astype(str)
client_df['state_l'] = client_df['State'].apply(lambda x: x.lower())
states = client_df['state_l'].unique().tolist()
state_dict = {'state': states}
state_df = pd.DataFrame(state_dict)
state_df['state_oid'] = state_df.index
new_state_dict = dict(zip(state_df['state'], state_df['state_oid']))
client_df['State'] = client_df['state_l'].map(new_state_dict)

# Drop the lowercase columns
client_df.drop(columns=['tract_l', 'county_l', 'state_l'], inplace=True)


## Create GA Intake Table

In [ ]:
ga_intake_df = data_df[['IntakeID',
                       'FolderID',
                       'UniqueID',
                       'GA_ACRES',
                       'GA GT', # this is going to be GA total isn't it? Or does it stay as GA estimate of current GT (total minus what client says is future)
                       'Total GT', # This will be the conservative total between client and GA numbers
                       'Notes'
                       ]].copy()

# Lowercase the ga_intake_df column names
ga_intake_df.columns = [x.lower().replace(' ', '_') for x in ga_intake_df.columns]
ga_intake_df.rename(columns={'uniqueid': 'id'},inplace=True)
ga_intake_df = ga_intake_df[['id', 'intakeid', 'folderid', 'ga_acres', 'ga_gt', 'total_gt', 'notes']]


## Create Status Table

In [ ]:
status_df = data_df[['UniqueID',
                    'Guidance',
                    'Expiration',
                    'GuidanceProvider',
                    'Status',
                    'Completed Date'
                    ]].copy()

## Create TicketID -> UniqueID lookup table

In [ ]:
ticketid_lookup_df = data_df[['UniqueID', 'TicketID']].copy() # the uniqueid will the the unique id for this table


In [ ]:
ticketid_list = data_df['TicketID'].unique().tolist()
ticketid_dict = {'ticketid': ticketid_list}
ticketid_df = pd.DataFrame(ticketid_dict)
ticketid_df['ticketid_oid'] = ticketid_df.index
new_ticketid_dict = dict(zip(ticketid_df['ticketid'], ticketid_df['ticketid_oid']))
ticketid_lookup_df['ticketid'] = ticketid_lookup_df['TicketID'].map(new_ticketid_dict)

ticketid_lookup_df.rename(columns={'TicketUID': 'id',
                                    'UniqueID': 'activity_id'}, inplace=True)

## Create the TCS lookup


In [ ]:
tcs_df = data_df.groupby('TicketID').agg({'Tract': 'first',
                                               'CMP': 'first',
                                               'Stand': ', '.join}).reset_index()

def split_stands(x):
    bugger_splot = list(set(x.split(', ')))
    bugger_splot.sort()
    return ", ".join(bugger_splot)
    
tcs_df['stands'] = tcs_df['Stand'].apply(split_stands)
tcs_df['comp'] = tcs_df['CMP'].apply(lambda x: x if x!="NP" else "")
tcs_df['TCS'] = tcs_df['Tract'] + "_" + tcs_df['comp'] + "_" + tcs_df['stands']
tcs_df['TCS'] = tcs_df['TCS'].apply(lambda x: x.replace("__", "_"))
tcs_df['tcs_oid'] = tcs_df.index
tcs_df = tcs_df[['tcs_oid', 'TicketID', 'TCS']].copy()

TicketID table should have a link to this

In [ ]:
tcs_mapper = dict(zip(tcs_df['TicketID'], tcs_df['tcs_oid']))
ticketid_df['tcs'] = ticketid_df['ticketid'].map(tcs_mapper)

In [ ]:
ticketid_df[ticketid_df['tcs'].isna()]

## Place the above tables into the Database
- [X] client_df
- [X] ga_intake_df
- [X] origin_df
- [X] county_df
- [X] species_df
- [X] state_df
- [X] status_df
- [X] tcs_df
- [X] ticketid_df
- [X] ticketid_lookup_df
- [X] tract_df

### Lowercase all of the column names

In [ ]:
# Client table
## lowercase client_df field names
client_df.columns = [x.lower() for x in client_df.columns]
client_df.columns = [x.replace(' ', '_') for x in client_df.columns]
client_df['id'] = client_df['uniqueid']
client_df.drop(columns=['uniqueid'],inplace=True)
client_df = client_df[['id', 'tract', 'cmp', 'stand', 'acres', 'age', 'county', 'state', 'stand_type', 'origin', 'scheduled', 'preba', 'pretpa', 'pregta',
                       'plnba', 'plntpa', 'plngta', 'client_gt', 'future_gt', 'reporting_period']]

# ga_intake
ga_intake_df.columns = [x.lower().replace(' ', '_') for x in ga_intake_df.columns]
ga_intake_df.rename(columns={'uniqueid': 'id'},inplace=True)
ga_intake_df = ga_intake_df[['id', 'intakeid', 'folderid', 'ga_acres', 'ga_gt', 'total_gt']]

# origin_df
origin_df.rename(columns={'OriginUID': 'id',
                          'Origin': 'origin'}, inplace=True)

# county_df
county_df.columns = [x.lower() for x in county_df.columns]

# species_df
species_df.columns = [x.lower() for x in species_df.columns]

# state_df
state_df.columns = [x.lower() for x in state_df.columns]

# status_df
status_df.columns = [x.lower() for x in status_df.columns]

# tcs_df
tcs_df.columns = [x.lower() for x in tcs_df.columns]

# ticketid_df
ticketid_df.columns = [x.lower() for x in ticketid_df.columns]

# ticketid_lookup_df
ticketid_lookup_df.columns = [x.lower() for x in ticketid_lookup_df.columns]

# tract_df
tract_df.columns = [x.lower() for x in tract_df.columns]



In [ ]:
state_df

# Mill Ticket sections

In [ ]:
projs = ['WAR', 'WRR', 'BJT']

In [ ]:
mill_loc = r"P:\Mill Tickets"
mega_df = pd.DataFrame()

for proj in projs:
    file_pattern = f"{proj}_RP*_Compiled.xlsm"
    big_df = pd.DataFrame()

    mill_name = []
    for root, dirs, files in os.walk(mill_loc):
        for file in files:
            if f"{proj}_RP" in file and file.endswith("Compiled.xlsm"):
                if "$" in file:
                    pass
                else:
                    print(root)
                    print(file)
                    df = pd.read_excel(os.path.join(root, file), sheet_name='RP_Project')
                    big_df = pd.concat([big_df, df])
    mega_df = pd.concat([mega_df, big_df])
                    
'''
    big_df.loc[big_df['Tons (U.S)'].isnull(), 'Tons (U.S)'] = big_df.loc[big_df['Tons (U.S)'].isnull(), 'TonsUS']
    big_df['Cull Tons'].fillna(big_df['CullTons'], inplace=True)
    big_df['Accepted Tons'].fillna(big_df['AcceptedTons'], inplace=True)
    big_df['Reporting Period'].fillna(big_df['ReportingPeriod'],inplace=True)

    def rep_fix(x):
        if "RP" in str(x):
            return x[-1]
        else:
            return x
        
    big_df['Reporting Period'] = big_df['Reporting Period'].apply(lambda x: rep_fix(x))
    big_df['Reporting Period'] = pd.to_numeric(big_df['Reporting Period'])

    big_df['Date'] = pd.to_datetime(big_df['Date'])

    harvest_df = big_df[['Date',
                    'Ticket',
                    'Mill',
                    'Product',
                    'Tons (U.S)',
                    'Cull Tons',
                    'Accepted Tons',
                    'Source',
                    'Notes',
                    'Reporting Period',
                    'TicketID',
                    'Week']].copy()

    harvest_df = harvest_df.drop_duplicates()
    '''

In [ ]:
mega_df.loc[mega_df['Tons (U.S)'].isnull(), 'Tons (U.S)'] = mega_df.loc[mega_df['Tons (U.S)'].isnull(), 'TonsUS']
mega_df['Cull Tons'].fillna(mega_df['CullTons'], inplace=True)
mega_df['Accepted Tons'].fillna(mega_df['AcceptedTons'], inplace=True)
mega_df['Reporting Period'].fillna(mega_df['ReportingPeriod'],inplace=True)

def rep_fix(x):
    if "RP" in str(x):
        return x[-1]
    else:
        return x
    
mega_df['Reporting Period'] = mega_df['Reporting Period'].apply(lambda x: rep_fix(x))
mega_df['Reporting Period'] = pd.to_numeric(mega_df['Reporting Period'])

mega_df['Date'] = pd.to_datetime(mega_df['Date'])

harvest_df = mega_df[['Date',
                'Ticket',
                'Mill',
                'Product',
                'Tons (U.S)',
                'Cull Tons',
                'Accepted Tons',
                'Source',
                'Notes',
                'Reporting Period',
                'TicketID',
                'Week']].copy()

harvest_df = harvest_df.drop_duplicates()
harvest_df.loc[harvest_df['TicketID'] == 'WAR18051XXXX', 'TicketID'] = 'WAR180518XXXX' # Found some mislabeled tickets

In [ ]:
mill_list = harvest_df['Mill'].unique().tolist()
mill_dict = {}
mill_dict['mill'] = mill_list
mill_df = pd.DataFrame(mill_dict)
mill_df['mill_uid'] = mill_df.index
mill_mapper = dict(zip(mill_df['mill'], mill_df['mill_uid']))
harvest_df['mill'] = harvest_df['Mill'].map(mill_mapper)
harvest_df.drop(columns=['Mill'], inplace=True)

In [ ]:
harvest_df['Week'] = harvest_df['Week'].astype(str)
harvest_df.loc[harvest_df['Week'] == 'Week3b', 'Week'] = "3"
def weeker(week):
    if "Week" in week:
        x = week.split("k")[1]
        x = int(x)
        return x
    elif "nan" in week:
        return 0
    else:
        return int(week)
    
harvest_df['Week_trial'] = harvest_df['Week'].apply(lambda x: weeker(x))

week_list = harvest_df['Week_trial'].unique().tolist()
week_dict = {}
week_dict['week'] = week_list
week_df = pd.DataFrame(week_dict)
week_df['week_uid'] = week_df.index
week_mapper = dict(zip(week_df['week'], week_df['week_uid']))
harvest_df['week'] = harvest_df['Week_trial'].map(week_mapper)

In [ ]:
product_list = harvest_df['Product'].unique().tolist()
product_dict = {}
product_dict['product'] = product_list
product_df = pd.DataFrame(product_dict)
product_df['product_uid'] = product_df.index
product_mapper = dict(zip(product_df['product'], product_df['product_uid']))
harvest_df['product'] = harvest_df['Product'].map(product_mapper)

In [ ]:
ticket_id_mapper = dict(zip(ticketid_df['ticketid'], ticketid_df['ticketid_oid']))
harvest_df['ticketid'] = harvest_df['TicketID'].map(ticket_id_mapper)
harvest_df['ticketid'] = harvest_df['ticketid'].astype(int)
harvest_df.drop(columns=['Product', 'TicketID', 'Week'],inplace=True)
harvest_df.rename(columns={'Tons (U.S)': 'total_tons'})

## Time to make the Expected Tons / RP table


My plan:
- Get the previous RP expected simply from MSS 
- Current RP pull from files I made for Rob

Shoot. An issue that I just realized is that the expected needs to be at the unique id. And I aligned the current rps on the ticket id to make it easier for Rob.

In [ ]:
# The following should get me up to RP8 for WAR
war_expected_inprog = data_df[(~data_df['Reporting Period'].isna()) & (data_df['UniqueID'].str.contains("WAR"))].copy()
war_expected_inprog = war_expected_inprog[war_expected_inprog['Reporting Period'] != 9]
war_expected_inprog = war_expected_inprog[['TicketID', 'Reporting Period', 'Current RP Expected GT']]
war_expected_inprog = war_expected_inprog[war_expected_inprog['Current RP Expected GT'] != ""]

In [ ]:
war_expected_inprog.head()

In [ ]:
war_expected_rp9 = pd.read_excel(r"./WAR_RP9_InProcess/Excel_sheets/rp9_investigate_matchesRob.xlsx", sheet_name='Sheet2')
war_expected_rp9.rename(columns=({'RP9 Expected': 'Current RP Expected GT'}), inplace=True)

In [ ]:
war_expected_rp9['Reporting Period'] = 9
war_expected_rp9, war_expected_rp9['Current RP Expected GT'].sum()